In [7]:
import tkinter as tk
from tkinter import ttk, messagebox
import csv
from PIL import ImageTk, Image
import requests
from io import BytesIO

In [11]:
import pandas as pd

data = pd.read_csv("./dataset/recipes_entrenamiento.csv")

In [4]:
keywords_dir = '../dataset/tags_unique.csv'
ingredients_dir = '../dataset/tags_unique.csv' # lo mismo porque aun no hay csv ingredientes

In [13]:
data2 =data.iloc[10:20]

In [15]:
def corregir(links, i = False):
    if links.startswith("c("):
        links = links[2:-1]
    links = links.replace('"', '')
    links = links.replace("'", '')
    links = links.replace("[", '')
    links = links.replace("]", '')
    if i:
        links = links.replace(', ',',\n')
        links = links.split(',')
    else:
        links = links.split(', ')
    # a veces vienen enlaces con regalitos antes del https por lo que hay que eliminar eso
    #links = [corregir_link(link) for link in links]
    return links

In [17]:
class FoodRecommenderApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Recomendador de Comidas")
        self.root.geometry("840x450+300+150")  # Tamaño inicial mayor
        
        # Variables de control
        self.calories = tk.StringVar()
        self.selected_keywords = []
        self.selected_ingredients = []
        
        # Cargar datos
        self.keywords = self.load_keywords_safe()
        self.ingredients_list = self.load_ingredients_safe()
        
        # Configurar interfaz
        self.setup_ui()
        self.show_main_page()

    def setup_ui(self):
        # Configurar estilo
        self.style = ttk.Style()
        self.style.configure('TButton', font=('Arial', 10))
        
        # Crear marcos principales
        self.main_page = ttk.Frame(self.root)
        self.results_page = ttk.Frame(self.root)
        
        # Configurar scroll para página principal
        self.setup_main_page_scroll()
        
        # Configurar página de resultados
        self.create_results_frame()
        ttk.Button(self.results_page, text="Cambiar Parámetros", 
                  command=self.show_main_page).pack(pady=10)

    def setup_main_page_scroll(self):
        # Canvas y scrollbar principal
        self.main_canvas = tk.Canvas(self.main_page)
        scrollbar = ttk.Scrollbar(self.main_page, orient="vertical", command=self.main_canvas.yview)
        self.scrollable_main = ttk.Frame(self.main_canvas)
        
        # Configurar eventos
        self.scrollable_main.bind("<Configure>", 
            lambda e: self.main_canvas.configure(scrollregion=self.main_canvas.bbox("all")))
        
        self.main_canvas.create_window((0, 0), window=self.scrollable_main, anchor="nw")
        self.main_canvas.configure(yscrollcommand=scrollbar.set)
        
        # Ajuste responsivo de ancho
        self.main_canvas.bind("<Configure>", lambda e: self.scrollable_main.config(width=e.width))
        
        scrollbar.pack(side="right", fill="y")
        self.main_canvas.pack(side="left", fill="both", expand=True)
        
        # Crear contenido desplazable
        self.create_input_sections()

    def create_input_sections(self):
        # Sección de calorías
        calories_frame = ttk.Frame(self.scrollable_main)
        ttk.Label(calories_frame, text="Calorías:").pack(side="left")
        ttk.Entry(calories_frame, textvariable=self.calories, width=15).pack(side="left", padx=5)
        calories_frame.pack(pady=10, anchor="w")
        
        # Secciones scrollables
        self.create_scrollable_subsections()

    def create_scrollable_subsections(self):
        # Contenedor principal para secciones
        sections_container = ttk.Frame(self.scrollable_main)
        sections_container.pack(fill="both", expand=True)
        
        # Keywords
        keywords_frame = ttk.LabelFrame(sections_container, text="Keywords")
        self.create_scrollable_section(keywords_frame, self.keywords, "keyword")
        keywords_frame.pack(side="left", fill="both", expand=True, padx=5, pady=5)
        
        # Ingredientes
        ingredients_frame = ttk.LabelFrame(sections_container, text="Ingredientes")
        self.create_scrollable_section(ingredients_frame, self.ingredients_list, "ingredient")
        ingredients_frame.pack(side="left", fill="both", expand=True, padx=5, pady=5)
        
        # Botón de búsqueda
        ttk.Button(self.scrollable_main, text="Buscar", command=self.safe_search).pack(pady=15)

    def create_scrollable_section(self, parent, items, section_type):
        # Canvas interno para cada sección
        canvas = tk.Canvas(parent, height=300)  # Altura fija
        scrollbar = ttk.Scrollbar(parent, orient="vertical", command=canvas.yview)
        scroll_frame = ttk.Frame(canvas)
        
        scroll_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
        canvas.create_window((0, 0), window=scroll_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        scrollbar.pack(side="right", fill="y")
        canvas.pack(side="left", fill="both", expand=True)
        
        # Botones check
        for i, item in enumerate(items):
            var = tk.BooleanVar()
            btn = ttk.Checkbutton(
                scroll_frame,
                text=item,
                variable=var,
                command=lambda v=var, it=item: self.update_selection(v, it, section_type),
                style='Toolbutton',
                width=25
            )
            btn.grid(row=i, column=0, sticky="w", padx=5, pady=2)

    def show_main_page(self):
        self.results_page.pack_forget()
        self.main_page.pack(fill="both", expand=True)
        self.root.minsize(840, 450)  # Tamaño mínimo

    def show_results_page(self):
        self.main_page.pack_forget()
        self.results_page.pack(fill="both", expand=True)
        self.root.minsize(840, 450)

    def load_keywords_safe(self):
        try:
            with open(keywords_dir, 'r', encoding='utf-8') as f:
                next(f)
                return list(row[0] for row in csv.reader(f) if row)#[:100]  # Limitar a 100 elementos
        except Exception as e:
            messagebox.showerror("Error", f"Error cargando keywords: {str(e)}")
            return []

    def load_ingredients_safe(self):
        try:
            with open(ingredients_dir, 'r', encoding='utf-8') as f:
                next(f)
                return list(row[0] for row in csv.reader(f) if row)#[:100]  # Limitar a 100 elementos
        except Exception as e:
            messagebox.showerror("Error", f"Error cargando ingredientes: {str(e)}")
            return []

    def update_selection(self, variable, item, selection_type):
        target_list = self.selected_keywords if selection_type == "keyword" else self.selected_ingredients
        if variable.get():
            target_list.append(item)
        else:
            if item in target_list:
                target_list.remove(item)

    def display_image(self, parent_frame, url):
        try:
            response = requests.get(url, timeout=10)
            img = Image.open(BytesIO(response.content))
            img.thumbnail((250, 250))  # Tamaño ajustable
            photo = ImageTk.PhotoImage(img)
            
            # Crear label dentro del contenedor padre
            label = ttk.Label(parent_frame, image=photo)
            label.image = photo  # Mantener referencia
            label.pack(side="left", padx=5, pady=5)
            
            # Guardar referencia en la lista
            self.image_references.append(photo)
            return True
            
        except Exception as e:
            print(f"Error cargando imagen: {str(e)}")
            return False

    def create_results_frame(self):
        # Marco principal de resultados
        results_container = ttk.Frame(self.results_page)
        results_container.pack(fill="both", expand=True)
        
        # Canvas y scrollbar
        self.results_canvas = tk.Canvas(results_container, highlightthickness=0)
        scrollbar = ttk.Scrollbar(results_container, orient="vertical", command=self.results_canvas.yview)
        self.scrollable_results = ttk.Frame(self.results_canvas)
        
        self.scrollable_results.bind("<Configure>", 
            lambda e: self.results_canvas.configure(scrollregion=self.results_canvas.bbox("all")))
        
        self.results_canvas.create_window((0, 0), window=self.scrollable_results, anchor="nw")
        self.results_canvas.configure(yscrollcommand=scrollbar.set)
        
        scrollbar.pack(side="right", fill="y")
        self.results_canvas.pack(side="left", fill="both", expand=True)
        
        # Contenedor para resultados
        self.results_container = ttk.Frame(self.scrollable_results)
        self.results_container.pack(fill="both", expand=True)
        
        # Lista para referencias de imágenes
        self.image_references = []

    def safe_search(self):
        try:
            # Validar datos mínimos
            if not self.calories.get().isdigit():
                messagebox.showerror("Error", "Ingrese un valor numérico para calorías")
                return
                
            # Limpiar resultados anteriores
            for widget in self.results_container.winfo_children():
                widget.destroy()
            self.image_references = []
            
            
            data2 #= [...]   Aquí la lógica real para obtener datos
            
            # Mostrar página de resultados
            self.show_results_page()
            
            # Procesar y mostrar resultados
            for i in range(len(data2)):
                # Procesar datos con tu función corregir()
                nombre = data2.iloc[i]['Name']
                links = corregir(data2.iloc[i]['Images'])[:2]  # Convertir a lista y tomar primeros 2
                instrucciones = corregir(data2.iloc[i]['RecipeInstructions'], True)
                
                # Crear sección por receta
                recipe_frame = ttk.Frame(self.results_container)
                
                # Nombre del platillo
                ttk.Label(recipe_frame, 
                        text=nombre, 
                        font=('Arial', 12, 'bold')).pack(pady=5)
                
                # Separador
                ttk.Separator(recipe_frame, orient='horizontal').pack(fill='x', pady=5)
                
                # Contenedor para imágenes
                img_container = ttk.Frame(recipe_frame)
                for link in links:
                    if link != "character(0)":
                        self.display_image(img_container, link)  # Modificamos display_image
                img_container.pack(pady=5)
                
                # Separador
                ttk.Separator(recipe_frame, orient='horizontal').pack(fill='x', pady=5)
                
                # Instrucciones
                instrucciones_text = "".join(instrucciones) if isinstance(instrucciones, list) else str(instrucciones)
                instruction_container = ttk.Frame(recipe_frame)
                instruction_container.pack(fill="x", pady=5, padx=10)

                ttk.Label(instruction_container,
                        text=instrucciones_text,
                        wraplength=500,  # Ajusta este valor según necesidad
                        anchor="w",
                        justify="left",
                        font=('Arial', 9)).pack(fill="x", side="left")
                
                # Separador final
                ttk.Separator(recipe_frame, orient='horizontal').pack(fill='x', pady=10)
                
                recipe_frame.pack(fill="x", pady=10)

        except Exception as e:
            messagebox.showerror("Error", f"Error en búsqueda: {str(e)}")
            
if __name__ == "__main__":
    root = tk.Tk()
    app = FoodRecommenderApp(root)
    root.mainloop()